# Caratterizzazione I_DC, P_Pump (f_pump FIXED)
### La scelta della frequenza della pompa è dettata dalle misure in "13_03_freq_4.h5"

In [ ]:
import h5py
import numpy as np
from h5utils import *
import matplotlib.pyplot as plt
from matplotlib import cm
import sys

### Dati:
- 13_03_DC_PUMP.h5
- 15_03_Lungo_2.h5

In [ ]:
filename = "../INTERESSANTI/13_03_DC_PUMP.h5";   # Change HERE!!!

Here you can see the content of the file

In [ ]:
"""PRINT ALL the .h5 file directories"""
h5dump(filename,group = "/");

In [ ]:
"""Get measure parameters"""
params_names, params_values = getDatasetValue(filename,"/",0)

# Print [Check if the order of paramerers is the same of the assegnation below!]
for i in np.arange(0,len(params_names)):
    print(str(params_names[i].decode("utf-8")),": ",str(params_values[i].decode("utf-8")))

# Vna
powerdBm = params_values[0] #[dBm]
navgs = params_values[3]
# Pump
freq_start = float(params_values[1])/1e9 #[GHz]
freq_stop = float(params_values[2])/1e9 #[Ghz]
ampl_min = float(params_values[3]) #[dBm]
ampl_max = float(params_values[4]) #[dBm]
# Bias
volt_min = float(params_values[5]) #[V]
volt_max = float(params_values[6]) #[V]
R = float(params_values[len(params_values)-1])
# You could retrieve current from resistance

# Points densities
n_freq = int(params_values[7])
n_ampl = int(params_values[8])
n_volt = int(params_values[9])

freqs = np.linspace(freq_start, freq_stop, n_freq) 
ampls = np.linspace(ampl_min, ampl_max, n_ampl)
volts = np.linspace(volt_min, volt_max, n_volt)
#volts = [0.227, 0.277, 0.327, 0.377, 0.427]

"""Print parameters arrays"""
print("\n\nIndex:\n")
temp_str = ""
volt_cifers = 3
for i in np.arange(0,n_volt):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(volts[i], volt_cifers)) + "; "
print("Bias Voltage (V): ",temp_str)
temp_str = ""
for i in np.arange(0,n_volt):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(volts[i]/R*1e3, volt_cifers)) + "; "
print("Bias Current (mA): ",temp_str)

temp_str = ""
freq_cifers = 3
for i in np.arange(0,n_freq):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(freqs[i], freq_cifers)) + "; "
print("Pump frequency (GHz): ",temp_str)

temp_str = ""
ampl_cifers = 3
for i in np.arange(0,n_ampl):
    temp_str = temp_str + "[" + str(i) + "] " + str(round(ampls[i], ampl_cifers)) + "; "
print("Pump power (dBm): ",temp_str)

Find optimal parameters among *ALL* the measurements in the .h5 file

In [ ]:
"""BEST PARAMETERS"""

fmin = 5e9 #[GHz]
fmax = 9e9 #[GHz]
fspan = fmax - fmin


# Get 'Off' measurement (will be subtracted to all measurements)
xoff, yoff = getDatasetValue(filename, "/Off", 0)
#group = "/dc_" + "0" + "/pumpfreq_" + "0" + "/pumppow_" + "0"
#xoff, yoff = getDatasetValue(filename, group, 0)
#yoff = np.zeros(len(yoff))

# Check if the frequency span [fmin,fmax] is valid
if xoff[0] <= fmin and xoff[len(xoff)-1] >= fmax:
    print("Band: [",round(fmin/1e9,freq_cifers),", ",round(fmax/1e9,freq_cifers),"] GHz")
else:
    sys.exit("Invalid frequency span!")
# Get i_min, i_max for the defined band
df = xoff[1] - xoff[0]
i_min = round((fmin - xoff[0])/df)
i_max = round(len(xoff) - 1 - (xoff[len(xoff)-1] - fmax)/df)
n_pts = i_max - i_min

band_averages = np.zeros((len(volts),len(freqs),len(ampls)))
band_std = np.zeros((len(volts),len(freqs),len(ampls)))
band_slope = np.zeros((len(volts),len(freqs),len(ampls)))
band_slope_err = np.zeros((len(volts),len(freqs),len(ampls)))
max_average = [0,0,0]    # Indexes of params for best band average
for k in np.arange(0,len(volts)):
    for j in np.arange(0,len(ampls)):
        for i in np.arange(0,len(freqs)):
            # Get desired dataset
            group = "/dc_" + str(k) + "/pumpfreq_" + str(i) + "/pumppow_" + str(j)
            try:
                temp_xs, temp_ys = getDatasetValue(filename, group, 0)
                # Remove the 'Off' measurement
                temp_ys = temp_ys - yoff
                # Focus on the defined BAND
                xs = temp_xs[i_min:i_max]
                ys = temp_ys[i_min:i_max]

                band_averages[k,i,j] = np.mean(ys)
                band_std[k,i,j] = np.std(ys)
                p, res, _, _, _ = np.polyfit(xs/1e9,ys,1,full=True)
                band_slope[k,i,j] = p[0]
                band_slope_err[k,i,j] = res/len(xs)
                if band_averages[max_average[0],max_average[1],max_average[2]] < band_averages[k,i,j]:
                    max_average = [k,i,j]
            except:
                print(volts[k],"   ",ampls[j])

# Get params for maximum band average
best_voltage = volts[max_average[0]]
best_freq = freqs[max_average[1]]
best_pow = ampls[max_average[2]]
print("Best params: ",best_voltage,"V, ",best_freq,"GHz, ",best_pow,"dBm")
print("Best Band average gain: ",round(band_averages[max_average[0],max_average[1],max_average[2]],3),"dBm")

# Plot band average varying only Pump frequency (other two params fixed at 'optimal' values)
fig_freq, ax_freq = plt.subplots()
ax_freq.plot(freqs, band_averages[max_average[0],:,max_average[2]],'.-')
ax_freq.set_title("Band average ["+str(round(fmin/1e9,freq_cifers))+", "+str(round(fmax/1e9,freq_cifers))+"] GHz\nBias: "+str(round(best_voltage,volt_cifers))+"V; Pump Pow: "+str(round(best_pow,ampl_cifers))+"dBm")
ax_freq.set_xlabel("Pump frequency [GHz]")
ax_freq.set_ylabel("Averaged gain [dB]")

# Plot band average varying only Pump power (other two params fixed at 'optimal' values)
fig_pow, ax_pow = plt.subplots()
ax_pow.plot(ampls, band_averages[max_average[0],max_average[1],:],'.-')
ax_pow.set_title("Band average ["+str(round(fmin/1e9,freq_cifers))+", "+str(round(fmax/1e9,freq_cifers))+"] GHz\nBias: "+str(round(best_voltage,volt_cifers))+"V; Pump Freq: "+str(round(best_freq,freq_cifers))+"GHz")
ax_pow.set_xlabel("Pump power [dBm]")
ax_pow.set_ylabel("Averaged gain [dB]")

# Plot band average varying only Bias voltage (other two params fixed at 'optimal' values)
fig_freq, ax_freq = plt.subplots()
ax_freq.plot(volts, band_averages[:,max_average[1],max_average[2]],'.-')
ax_freq.set_title("Band average ["+str(round(fmin/1e9,freq_cifers))+", "+str(round(fmax/1e9,freq_cifers))+"] GHz\nPump Freq: "+str(round(best_freq,freq_cifers))+"GHz; Pump Pow: "+str(round(best_pow,ampl_cifers))+"dBm")
ax_freq.set_xlabel("Bias voltage [V]")
ax_freq.set_ylabel("Averaged gain [dB]")

In [ ]:
"""3D PLOT"""
from mpl_toolkits.mplot3d import Axes3D


best_gain = round(band_averages[max_average[0],max_average[1],max_average[2]],3)
# In order to have a more precise colormap, we exclude values less than a
# selected one, assigning them to this selected value
minimum_plot_gain = 3 #[dBm]


B, D = np.meshgrid(ampls, volts)
average = np.zeros((len(ampls),len(volts)))
std = np.zeros((len(ampls),len(volts)))
slope = np.zeros((len(ampls),len(volts)))
slope_err = np.zeros((len(ampls),len(volts)))
for x in np.arange(0,len(ampls)):
    for y in np.arange(0,len(volts)):
        if band_averages[y,0,x] < minimum_plot_gain:
            average[x,y] = minimum_plot_gain
        else:
            average[x,y] = band_averages[y,0,x]
        std[x,y] = abs(band_std[y,0,x]/band_averages[y,0,x])
        slope[x,y] = abs(band_slope[y,0,x])
        slope_err[x,y] = abs(band_slope_err[y,0,x])

D = D/R*1e3 # Voltage [V] to current [mA]
fig = plt.figure()
plt.pcolor(D, B, np.transpose(average), vmin=minimum_plot_gain, vmax=best_gain)
plt.colorbar()
plt.ylabel('Pump Power [dBm]')
plt.xlabel('DC current [mA]')
plt.title('Average Gain [dBm] in [5,8]GHz\nf_pump = 15.722 GHz')
plt.show()

fig2 = plt.figure()
plt.pcolor(D, B, np.transpose(slope), vmax=2)
plt.colorbar()
plt.ylabel('Pump Power [dBm]')
plt.xlabel('DC current [mA]')
plt.title('Band Slope in [5,8]GHz\nf_pump = 15.722 GHz')
plt.show()

fig3 = plt.figure()
plt.pcolor(D, B, np.transpose(slope_err),vmax=3)
plt.colorbar()
plt.ylabel('Pump Power [dBm]')
plt.xlabel('DC current [mA]')
plt.title('Band Slope Fit error in [5,8]GHz\nf_pump = 15.722 GHz')
plt.show()

fig4 = plt.figure()
plt.pcolor(D, B, np.transpose(std),vmin=0,vmax=0.5)
plt.colorbar()
plt.ylabel('Pump Power [dBm]')
plt.xlabel('DC current [mA]')
plt.title('Gain Std/Average in [5,8]GHz\nf_pump = 15.722 GHz')
plt.show()

In [ ]:
"""3D PLOT FULL INFO"""
# Retrieve vna frequencies and Off measure
frequencies, yoff = getDatasetValue(filename, "/Off", 0)
frequencies = frequencies/1e9
B, D = np.meshgrid(frequencies, ampls)
S21dB = np.zeros((len(frequencies),len(ampls)))
#print(len(S21dB[0]))

plt.set_cmap('Spectral')
#j = 86
for j in np.array([0]):
    for i in np.arange(0,len(ampls)):
        group = "/dc_" + str(j) + "/pumpfreq_0/pumppow_" + str(i)
        temp_xs, temp_ys = getDatasetValue(filename, group, 0)
        frequencies, yoff = getDatasetValue(filename, "/Off_"+str(j), 0)
        S21dB[:,i] = temp_ys - yoff
    
    plt.figure()
    plt.pcolor(B, D, np.transpose(S21dB), vmin=-12, vmax = 12)
    cbar = plt.colorbar()
    cbar.ax.set_ylabel("$|S_{21}|$ [dB]")
    plt.xlabel("VNA frequency [GHz]")
    plt.ylabel("Pump power [dBm]")
    plt.title("DC current " + str(round(volts[j]/R*1e3,3)) + " mA")
    plt.savefig("../cmaps/cmap2_" + str(j) + ".png")
    #plt.close()